In [ ]:
import torch

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("CUDA device count:", torch.cuda.device_count())
    print("CUDA device name:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available.")

# Fine-Tuning the Semantic Seach Model

***sentence-transformers/msmarco-bert-base-dot-v5***

https://huggingface.co/sentence-transformers/msmarco-bert-base-dot-v5

In [ ]:
import pandas as pd
import os

In [ ]:
# Define the core path
BASE_PATH = r'E:\Software\Data Science and AI\NLP\Edliyye\Legal Acts Question Answering\Data Collection'

text_path = os.path.join(BASE_PATH, 'finetune_data.parquet')

In [ ]:
df = pd.read_parquet(text_path)

display(df)
df.info()
df.index

In [ ]:
import torch
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, InputExample, losses
from tqdm.notebook import tqdm
from datasets import load_dataset

# Load the model
model = SentenceTransformer("sentence-transformers/msmarco-bert-base-dot-v5")

# Load and prepare the dataset
data_path = "/content/finetune_data.parquet"  # Path to your file
dataset = load_dataset("parquet", data_files=data_path)["train"]
train_samples = [InputExample(texts=[row["query"], row["text"]], label=float(row["label"])) for row in dataset]

# Custom collate function for DataLoader
def collate_fn(batch):
    sentence_pairs = [{"sentence1": example.texts[0], "sentence2": example.texts[1]} for example in batch]
    labels = torch.tensor([example.label for example in batch], dtype=torch.float32)
    return sentence_pairs, labels

# Create DataLoader with the custom collate function
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=10, collate_fn=collate_fn)

# Define the loss function
loss_function = losses.CosineSimilarityLoss(model)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training parameters
num_epochs = 5
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)
    for sentence_pairs, labels in progress_bar:
        optimizer.zero_grad()
        
        # Move labels to the appropriate device
        labels = labels.to(device)
        
        # Tokenize sentence pairs directly in the required format
        sentence_features = [
            {
                "input_ids": model.tokenizer(pair["sentence1"], pair["sentence2"], 
                                             padding=True, truncation=True, return_tensors="pt")["input_ids"],
                "attention_mask": model.tokenizer(pair["sentence1"], pair["sentence2"], 
                                                  padding=True, truncation=True, return_tensors="pt")["attention_mask"]
            }
            for pair in sentence_pairs
        ]

        # Convert sentence_features to a format compatible with loss function
        sentence_features = [{k: v.to(device) for k, v in feature.items()} for feature in sentence_features]
        
        # Calculate loss
        loss = loss_function(sentence_features, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        progress_bar.set_postfix(loss=total_loss / len(train_dataloader))

    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {total_loss / len(train_dataloader)}")

# Save the model
model.save("/content/drive/MyDrive/msmarco-bert-base-dot-v5-cosine/fine-tuned")

**Below is the modified code with the following changes:**

1. ***Lowered Learning Rate***: Set to 5e-6.
2. ***Increased Epochs***: Set to 10.
3. ***Gradient Accumulation***: Added accumulation for every 2 batches.
4. ***Layer-Wise Learning Rate Decay (LLRD)***: Applied a different learning rate for each layer, with lower layers having smaller learning rates.

In [ ]:
# !pip install sentence-transformers datasets torch

import torch
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, InputExample, losses
from tqdm.notebook import tqdm
from datasets import load_dataset

# Load the model
model = SentenceTransformer("sentence-transformers/msmarco-bert-base-dot-v5")

# Load and prepare the dataset
data_path = "/content/finetune_data.parquet"  # Path to your file (this one is for Colab)
dataset = load_dataset("parquet", data_files=data_path)["train"]
train_samples = [InputExample(texts=[row["query"], row["text"]], label=float(row["label"])) for row in dataset]

# Custom collate function for DataLoader
def collate_fn(batch):
    sentence_pairs = [{"sentence1": example.texts[0], "sentence2": example.texts[1]} for example in batch]
    labels = torch.tensor([example.label for example in batch], dtype=torch.float32)
    return sentence_pairs, labels

# Create DataLoader with the custom collate function and adjusted batch size
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=10, collate_fn=collate_fn)

# Define the loss function
loss_function = losses.CosineSimilarityLoss(model)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training parameters
num_epochs = 10
learning_rate = 5e-6
accumulation_steps = 2

# Apply Layer-Wise Learning Rate Decay (LLRD)
optimizer_grouped_parameters = [
    {
        "params": [param for name, param in model.named_parameters() if "encoder.layer.0" in name],
        "lr": learning_rate * 0.5
    },
    {
        "params": [param for name, param in model.named_parameters() if "encoder.layer.1" in name],
        "lr": learning_rate * 0.6
    },
    {
        "params": [param for name, param in model.named_parameters() if "encoder.layer.2" in name],
        "lr": learning_rate * 0.7
    },
    {
        "params": [param for name, param in model.named_parameters() if "encoder.layer.3" in name],
        "lr": learning_rate * 0.8
    },
    {
        "params": [param for name, param in model.named_parameters() if "encoder.layer.4" in name],
        "lr": learning_rate * 0.9
    },
    {
        "params": [param for name, param in model.named_parameters() if "encoder.layer.5" in name],
        "lr": learning_rate
    },
]

optimizer = torch.optim.AdamW(optimizer_grouped_parameters)

# Training loop with gradient accumulation and LLRD
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)
    for i, (sentence_pairs, labels) in enumerate(progress_bar):
        # Move labels to the appropriate device
        labels = labels.to(device)
        
        # Tokenize sentence pairs directly in the required format
        sentence_features = [
            {
                "input_ids": model.tokenizer(pair["sentence1"], pair["sentence2"], 
                                             padding=True, truncation=True, return_tensors="pt")["input_ids"],
                "attention_mask": model.tokenizer(pair["sentence1"], pair["sentence2"], 
                                                  padding=True, truncation=True, return_tensors="pt")["attention_mask"]
            }
            for pair in sentence_pairs
        ]

        # Convert sentence_features to a format compatible with loss function
        sentence_features = [{k: v.to(device) for k, v in feature.items()} for feature in sentence_features]
        
        # Calculate loss directly with sentence_features and labels
        loss = loss_function(sentence_features, labels) / accumulation_steps
        loss.backward()

        # Gradient accumulation step
        if (i + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        total_loss += loss.item() * accumulation_steps  # Correct scaling for accumulated loss
        progress_bar.set_postfix(loss=total_loss / len(train_dataloader))

    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {total_loss / len(train_dataloader)}")

# Save the model
model.save("/content/drive/MyDrive/msmarco-bert-base-dot-v5-cosine/fine-tuned")

```
Epoch 1/10 - Loss: 0.36621989842790825
Epoch 2/10 - Loss: 0.28407311846430483
Epoch 3/10 - Loss: 0.2611631113749284
Epoch 4/10 - Loss: 0.25623579483765824
Epoch 5/10 - Loss: 0.2572602228476451
Epoch 6/10 - Loss: 0.2544843180821492
Epoch 7/10 - Loss: 0.2549075830441255
Epoch 8/10 - Loss: 0.2535590521418131
Epoch 9/10 - Loss: 0.2545134897415455
Epoch 10/10 - Loss: 0.2544914065645291
```

## Inferencing the Model

In [ ]:
# Code from documentation in Hugging Face!

from sentence_transformers import SentenceTransformer, util

query = "cinayət təqibi nədir?"
docs = ["83.1-1. Həm Azərbaycan Respublikasının, həm də ərazisində törədildiyi xarici dövlətin qanunvericiliyinə əsasən cinayət sayılan əmələ görə xarici dövlətin məhkəmə hökmünə əsasən yaranmış məhkumluq barədə məlumat cinayət təqibinin gedişində daxil olduqda və müvafiq hökm Azərbaycan Respublikasında qanunla müəyyən edilmiş qaydada tanındıqda, bu məhkumluq da cinayətlərin residivi zamanı və cəza təyin edilərkən nəzərə alınır.", 
        "1.1. Azərbaycan Respublikasının cinayət-prosessual qanunvericiliyi cinayətin əlamətlərini əks etdirən əməllərin cinayət olub-olmamağını, cinayəti törətməkdə təqsirləndirilən şəxsin təqsirli olub-olmamağını, habelə cinayət qanunu ilə nəzərdə tutulmuş əməlləri törətməkdə şübhəli və ya təqsirləndirilən şəxsin cinayət təqibinin və müdafiəsinin, hüquqi şəxs barəsində cinayət-hüquqi tədbirlərin tətbiq edilməsinin hüquqi prosedurlarını müəyyən edir.", 
        "1.1. Bu Məcəllə inzibati hüquq münasibətləri ilə bağlı mübahisələrin (bundan sonra - inzibati mübahisələr) məhkəmə aidiyyətini, həmin mübahisələrə məhkəmədə baxılmasının və həll edilməsinin prosessual prinsiplərini və qaydalarını müəyyən edir. 1.2. Bu Məcəllə ilə başqa qayda müəyyən edilmədiyi və bu Məcəllə ilə nəzərdə tutulmuş prosessual prinsiplərə zidd olmadığı hallarda, inzibati mübahisələrə dair işlər üzrə məhkəmə icraatında (bundan sonra - inzibati məhkəmə icraatı) Azərbaycan Respublikasının Mülki Prosessual Məcəlləsinin müddəaları tətbiq oluna bilər. 1.3. Ələt azad iqtisadi zonasında inzibati hüquq münasibətləri ilə bağlı mübahisələrə baxılması “Ələt azad iqtisadi zonası haqqında” Azərbaycan Respublikası Qanununun tələblərinə uyğun olaraq tənzimlənir.", 
        "Maddə 1. Şəhərsalma və tikinti qanunvericiliyinin təyinatı Azərbaycan Respublikasının şəhərsalma və tikinti qanunvericiliyi şəhərsalma və tikinti fəaliyyətinin hüquqi əsaslarını, prinsiplərini, eləcə də dövlətin, bələdiyyələrin, fiziki və ya hüquqi şəxslərin şəhərsalma və tikinti fəaliyyəti sahəsində hüquq və vəzifələrini müəyyən edir."]

model_path = r"E:\Software\Data Science and AI\NLP\Edliyye\Legal Acts Question Answering\Fine-Tuning\msmarco-bert-base-dot-v5-cosine\final"
model = SentenceTransformer(model_path)

#Encode query and documents
query_emb = model.encode(query)
doc_emb = model.encode(docs)

#Compute dot score between query and all document embeddings
scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()

#Combine docs & scores
doc_score_pairs = list(zip(docs, scores))

#Sort by decreasing score
doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)

#Output passages & scores
print("Query:", query)
print('\n')
for doc, score in doc_score_pairs:
    print(score, doc)
    print('\n')

In [ ]:
# Custom Inference Code!

import torch
from sentence_transformers import SentenceTransformer, util

model_path = r"E:\Software\Data Science and AI\NLP\Edliyye\Legal Acts Question Answering\Fine-Tuning\msmarco-bert-base-dot-v5-cosine\final"
model = SentenceTransformer(model_path)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define a single query for testing
query = "cinayət təqibi nədir?"

# Define texts to compare with the query
texts = [
    "1.1. Azərbaycan Respublikasının cinayət-prosessual qanunvericiliyi cinayətin əlamətlərini əks etdirən əməllərin cinayət olub-olmamağını, cinayəti törətməkdə təqsirləndirilən şəxsin təqsirli olub-olmamağını, habelə cinayət qanunu ilə nəzərdə tutulmuş əməlləri törətməkdə şübhəli və ya təqsirləndirilən şəxsin cinayət təqibinin və müdafiəsinin, hüquqi şəxs barəsində cinayət-hüquqi tədbirlərin tətbiq edilməsinin hüquqi prosedurlarını müəyyən edir.",
    "83.1-1. Həm Azərbaycan Respublikasının, həm də ərazisində törədildiyi xarici dövlətin qanunvericiliyinə əsasən cinayət sayılan əmələ görə xarici dövlətin məhkəmə hökmünə əsasən yaranmış məhkumluq barədə məlumat cinayət təqibinin gedişində daxil olduqda və müvafiq hökm Azərbaycan Respublikasında qanunla müəyyən edilmiş qaydada tanındıqda, bu məhkumluq da cinayətlərin residivi zamanı və cəza təyin edilərkən nəzərə alınır.",
    "Maddə 1. Şəhərsalma və tikinti qanunvericiliyinin təyinatı Azərbaycan Respublikasının şəhərsalma və tikinti qanunvericiliyi şəhərsalma və tikinti fəaliyyətinin hüquqi əsaslarını, prinsiplərini, eləcə də dövlətin, bələdiyyələrin, fiziki və ya hüquqi şəxslərin şəhərsalma və tikinti fəaliyyəti sahəsində hüquq və vəzifələrini müəyyən edir.",
    "1.1. Bu Məcəllə inzibati hüquq münasibətləri ilə bağlı mübahisələrin (bundan sonra - inzibati mübahisələr) məhkəmə aidiyyətini, həmin mübahisələrə məhkəmədə baxılmasının və həll edilməsinin prosessual prinsiplərini və qaydalarını müəyyən edir. 1.2. Bu Məcəllə ilə başqa qayda müəyyən edilmədiyi və bu Məcəllə ilə nəzərdə tutulmuş prosessual prinsiplərə zidd olmadığı hallarda, inzibati mübahisələrə dair işlər üzrə məhkəmə icraatında (bundan sonra - inzibati məhkəmə icraatı) Azərbaycan Respublikasının Mülki Prosessual Məcəlləsinin müddəaları tətbiq oluna bilər. 1.3. Ələt azad iqtisadi zonasında inzibati hüquq münasibətləri ilə bağlı mübahisələrə baxılması “Ələt azad iqtisadi zonası haqqında” Azərbaycan Respublikası Qanununun tələblərinə uyğun olaraq tənzimlənir.",
    "7.0.4. cinayət təqibi — cinayət hadisəsinin müəyyən edilməsi, cinayət qanunu ilə nəzərdə tutulmuş əməli törətmiş şəxsin ifşa olunması, ittiham irəli sürülməsi, bu ittihamın məhkəmədə müdafiə edilməsi, ona cəza təyin edilməsi, zəruri olduqda prosessual məcburiyyət tədbirlərinin təmin edilməsi məqsədi ilə həyata keçirilən cinayət-prosessual fəaliyyətdir;"
]

# Function to compute similarity scores
def compute_similarity(query, texts):
    # Tokenize and embed query once
    query_embedding = model.encode(query, convert_to_tensor=True, device=device)
    
    results = []
    for text in texts:
        # Tokenize and embed each text
        text_embedding = model.encode(text, convert_to_tensor=True, device=device)
        
        # Compute cosine similarity
        cosine_score = util.cos_sim(query_embedding, text_embedding).item()
        
        # Collect the results
        results.append({
            "query": query,
            "text": text,
            "similarity_score": cosine_score
        })

    # Sort results by similarity score in descending order
    results = sorted(results, key=lambda x: x["similarity_score"], reverse=True)
    return results

# Run similarity computation
results = compute_similarity(query, texts)

# Display sorted results
for result in results:
    print(f"Query: {result['query']}")
    print(f"Text: {result['text']}")
    print(f"Similarity Score: {result['similarity_score']:.4f}")
    print("-" * 50)

- ***The test comparison has demonstrated that the fine-tuned model's performance is much more better than default (raw, pre-trained) one!***
- ***The model before fine-tuning is not able to distinguish similar sentences from those that have absolutely no common meaning with the user query and assigned all texts similarily high scores, whereas the fine-tuned model assigned >0.5 scores for relevant ones and <0.5 for irrelevant ones!***
- ***That is why, I can set the filter to 0.5 and display results above 0.5 in prompt!***

## Inference on the Sample n% of the Corpus

In [ ]:
import torch
from sentence_transformers import SentenceTransformer, util
import pandas as pd

# Paths to model and corpus file
model_path = r"E:\Software\Data Science and AI\NLP\Edliyye\Legal Acts Question Answering\Fine-Tuning\msmarco-bert-base-dot-v5-cosine\final"
corpus_path = r"E:\Software\Data Science and AI\NLP\Edliyye\Legal Acts Question Answering\Data Collection\corpus.parquet"

# Load the fine-tuned model
model = SentenceTransformer(model_path)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load the text data from the corpus file
corpus_df = pd.read_parquet(corpus_path)

# Sample n% of the corpus
sampled_corpus_df = corpus_df.sample(frac=0.01, random_state=42)
texts = sampled_corpus_df['text'].tolist()  # Extract the 'text' column as a list

# Define a single query for testing
query = "icra hərəkətləri neçə günə başa çatdırılmalıdır?"

# Define a threshold for similarity score (e.g., 0.5)
SIMILARITY_THRESHOLD = 0.5

# Function to compute similarity scores with a threshold
def compute_similarity(query, texts, threshold=SIMILARITY_THRESHOLD):
    # Tokenize and embed the query once
    query_embedding = model.encode(query, convert_to_tensor=True, device=device)
    
    results = []
    for text in texts:
        # Tokenize and embed each text
        text_embedding = model.encode(text, convert_to_tensor=True, device=device)
        
        # Compute cosine similarity
        cosine_score = util.cos_sim(query_embedding, text_embedding).item()
        
        # Collect results if they meet the threshold
        if cosine_score >= threshold:
            results.append({
                "query": query,
                "text": text,
                "similarity_score": cosine_score
            })

    # Sort results by similarity score in descending order
    results = sorted(results, key=lambda x: x["similarity_score"], reverse=True)
    return results

# Run similarity computation
results = compute_similarity(query, texts)

# Display sorted results that meet the threshold
for result in results:
    print(f"Query: {result['query']}")
    print(f"Text: {result['text']}")
    print(f"Similarity Score: {result['similarity_score']:.4f}")
    print("-" * 50)

## Inference on the Whole Corpus

In [ ]:
import torch
from sentence_transformers import SentenceTransformer, util
import pandas as pd

# Paths to model and corpus file
model_path = r"E:\Software\Data Science and AI\NLP\Edliyye\Legal Acts Question Answering\Fine-Tuning\msmarco-bert-base-dot-v5-cosine\final"
corpus_path = r"E:\Software\Data Science and AI\NLP\Edliyye\Legal Acts Question Answering\Data Collection\corpus.parquet"

# Load the fine-tuned model
model = SentenceTransformer(model_path)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load the text data from the corpus file
corpus_df = pd.read_parquet(corpus_path)
texts = corpus_df['text'].tolist()  # Extract the 'text' column as a list

# Define a single query for testing
query = "icra hərəkətləri neçə günə başa çatdırılmalıdır?"

# Define a threshold for similarity score (e.g., 0.5)
SIMILARITY_THRESHOLD = 0.5

# Function to compute similarity scores with a threshold
def compute_similarity(query, texts, threshold=SIMILARITY_THRESHOLD):
    # Tokenize and embed the query once
    query_embedding = model.encode(query, convert_to_tensor=True, device=device)
    
    results = []
    for text in texts:
        # Tokenize and embed each text
        text_embedding = model.encode(text, convert_to_tensor=True, device=device)
        
        # Compute cosine similarity
        cosine_score = util.cos_sim(query_embedding, text_embedding).item()
        
        # Collect results if they meet the threshold
        if cosine_score >= threshold:
            results.append({
                "query": query,
                "text": text,
                "similarity_score": cosine_score
            })

    # Sort results by similarity score in descending order
    results = sorted(results, key=lambda x: x["similarity_score"], reverse=True)
    return results

# Run similarity computation
results = compute_similarity(query, texts)

# Display sorted results that meet the threshold
for result in results:
    print(f"Query: {result['query']}")
    print(f"Text: {result['text']}")
    print(f"Similarity Score: {result['similarity_score']:.4f}")
    print("-" * 50)

## Batched Inference Instead of Loop

In [ ]:
import torch
from sentence_transformers import SentenceTransformer, util
import pandas as pd

# Paths to model and corpus file
model_path = r"E:\Software\Data Science and AI\NLP\Edliyye\Legal Acts Question Answering\Fine-Tuning\msmarco-bert-base-dot-v5-cosine\final"
corpus_path = r"E:\Software\Data Science and AI\NLP\Edliyye\Legal Acts Question Answering\Data Collection\corpus.parquet"

# Load the fine-tuned model
model = SentenceTransformer(model_path)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load the text data from the corpus file
corpus_df = pd.read_parquet(corpus_path)
texts = corpus_df['text'].tolist()  # Extract the 'text' column as a list

# Define a single query for testing
query = "icra hərəkətləri neçə günə başa çatdırılmalıdır?"

# Define a threshold for similarity score (e.g., 0.5)
SIMILARITY_THRESHOLD = 0.5

# Function to compute similarity scores with batching and optimized cosine similarity
def compute_similarity(query, texts, threshold=SIMILARITY_THRESHOLD):
    # Tokenize and embed the query once
    query_embedding = model.encode(query, convert_to_tensor=True, device=device)
    
    # Encode all texts in a single batch
    text_embeddings = model.encode(texts, convert_to_tensor=True, device=device)
    
    # Compute cosine similarities between the query and all texts at once
    cosine_scores = util.cos_sim(query_embedding, text_embeddings).cpu().numpy().flatten()
    
    # Collect results that meet the threshold
    results = [
        {"query": query, "text": texts[i], "similarity_score": score}
        for i, score in enumerate(cosine_scores) if score >= threshold
    ]

    # Sort results by similarity score in descending order
    results = sorted(results, key=lambda x: x["similarity_score"], reverse=True)
    return results

# Run similarity computation
results = compute_similarity(query, texts)

# Display sorted results that meet the threshold
for result in results:
    print(f"Query: {result['query']}")
    print(f"Text: {result['text']}")
    print(f"Similarity Score: {result['similarity_score']:.4f}")
    print("-" * 50)

## Pre-Computing and Saving the Embeddings

In [ ]:
import torch
from sentence_transformers import SentenceTransformer
import pandas as pd

# Paths to model and corpus file
model_path = r"E:\Software\Data Science and AI\NLP\Edliyye\Legal Acts Question Answering\Fine-Tuning\msmarco-bert-base-dot-v5-cosine\final"
corpus_path = r"E:\Software\Data Science and AI\NLP\Edliyye\Legal Acts Question Answering\Data Collection\corpus.parquet"

# Load the fine-tuned model
model = SentenceTransformer(model_path)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load the text data from the corpus file
corpus_df = pd.read_parquet(corpus_path)
texts = corpus_df['text'].tolist()  # Extract the 'text' column as a list

# Compute embeddings for all texts in batches
batch_size = 64  # Adjust batch size based on GPU memory availability
embeddings = []

for i in range(0, len(texts), batch_size):
    batch_texts = texts[i:i+batch_size]
    # Encode batch and move to the CPU for storage (detach from GPU)
    batch_embeddings = model.encode(batch_texts, convert_to_tensor=True, device=device).cpu()
    embeddings.extend(batch_embeddings.numpy())  # Convert to numpy and add to list

# Add embeddings as a new column to the DataFrame
corpus_df['embeddings'] = embeddings

# Save the modified DataFrame back to the same parquet file
corpus_df.to_parquet(corpus_path, index=False)
print("Embeddings have been precomputed and saved to the corpus file.")

In [ ]:
import torch
from sentence_transformers import SentenceTransformer
import pandas as pd
import time

# Paths to model and corpus file
model_path = "/content/drive/MyDrive/msmarco-bert-base-dot-v5-cosine/final"
corpus_path = "/content/drive/MyDrive/corpus.parquet"

# Load the fine-tuned model
model = SentenceTransformer(model_path)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load the text data from the corpus file
corpus_df = pd.read_parquet(corpus_path)
texts = corpus_df['text'].tolist()  # Extract the 'text' column as a list

# Compute embeddings for all texts in batches
batch_size = 32  # Adjust batch size based on GPU memory availability
embeddings = []

# Measure time for the process
start_time = time.time()

for i in range(0, len(texts), batch_size):
    batch_texts = texts[i:i+batch_size]
    # Encode batch and move to the CPU for storage (detach from GPU)
    batch_embeddings = model.encode(batch_texts, convert_to_tensor=True, device=device).cpu()
    embeddings.extend(batch_embeddings.numpy())  # Convert to numpy and add to list
    
    # Optionally log progress every few batches
    if (i // batch_size) % 10 == 0:
        print(f"Processed batch {i // batch_size} of {len(texts) // batch_size}")

end_time = time.time()
print(f"Total time for embedding computation: {end_time - start_time:.2f} seconds")

# Add embeddings as a new column to the DataFrame
corpus_df['embeddings'] = embeddings

# Save the modified DataFrame back to the same parquet file
corpus_df.to_parquet(corpus_path, index=False)
print("Embeddings have been precomputed and saved to the corpus file.")

# Inference Code

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import numpy as np
from numpy.linalg import norm
from tqdm import tqdm
tqdm.pandas()


# Paths to model and corpus file
model_path = r"E:\Software\Data Science and AI\NLP\Edliyye\Legal Acts Question Answering\Fine-Tuning\msmarco-bert-base-dot-v5-cosine\final"
corpus_path = r"E:\Software\Data Science and AI\NLP\Edliyye\Legal Acts Question Answering\Data Collection\corpus_embed.parquet"

# Load the fine-tuned model
model = SentenceTransformer(model_path).to("cuda")

data = pd.read_parquet(corpus_path)[["text", "embeddings"]]
data.info()

In [ ]:
cosine = lambda x, y: np.dot(x, y)/(norm(x)*norm(y))

model_vector = model.encode("İcra hərəkətlərinin həyata keçirilməsi müddətləri")

data["cosine"] = data["embeddings"].progress_apply(lambda x: cosine(x, model_vector))

s = data.loc[data["cosine"] >= 0.75][["cosine", "text"]].sort_values("cosine", ascending=False).head(20).reset_index(drop=True)
s

In [ ]:
s['text'].iloc[11]

In [ ]:
for i in s['text']:
    print(i)

## My Inference Code

In [ ]:
import torch
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np

# Paths to model and corpus file
model_path = r"E:\Software\Data Science and AI\NLP\Edliyye\Legal Acts Question Answering\Fine-Tuning\msmarco-bert-base-dot-v5-cosine\final"
corpus_path = r"E:\Software\Data Science and AI\NLP\Edliyye\Legal Acts Question Answering\Data Collection\corpus_embed.parquet"

# Load the fine-tuned model
model = SentenceTransformer(model_path)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load the embeddings, texts, urls, and names from the corpus file
corpus_df = pd.read_parquet(corpus_path)
embeddings = np.array(corpus_df['embeddings'].tolist())  # Assuming 'embeddings' column contains precomputed embeddings
texts = corpus_df['text'].tolist()  # Texts for reference
urls = corpus_df['url'].tolist()    # URLs for reference
names = corpus_df['name'].tolist()  # Names for reference

In [ ]:
# Define a single query for testing
query = "İcra hərəkətlərinin həyata keçirilməsi müddətləri"

# Define a threshold for similarity score
SIMILARITY_THRESHOLD = 0.75

# Function to compute similarity scores with filtering and limit results to top K
def compute_similarity(query, embeddings, threshold=SIMILARITY_THRESHOLD, top_k=20):
    # Embed the query once and convert to NumPy array
    query_embedding = model.encode(query, convert_to_tensor=False)

    # Compute cosine similarity for all embeddings in a single operation
    similarities = util.cos_sim(query_embedding, embeddings).cpu().numpy().flatten()

    # Collect results with filtering for unique texts
    results = []
    seen_texts = set()
    for i, score in enumerate(similarities):
        if score >= threshold and texts[i] not in seen_texts:
            seen_texts.add(texts[i])  # Track text to avoid duplicates
            results.append({
                "query": query,
                "text": texts[i],
                "similarity_score": score,
                "url": urls[i],
                "name": names[i]
            })

    # Sort results by similarity score in descending order and limit to top_k results
    results = sorted(results, key=lambda x: x["similarity_score"], reverse=True)[:top_k]
    return results

# Run similarity computation
results = compute_similarity(query, embeddings)

print(f"User Query: {result['query']}")
print()
print("Retrieved Text Passages:")
print()
# Display sorted results that meet the threshold
for result in results:
    print(f"Text: {result['text']}")
    print(f"URL: {result['url']}")
    print(f"Document Name: {result['name']}")
    print("-" * 50)

In [ ]:
corpus_df.loc[corpus_df['text'].str.contains("Maddə 12. İcra hərəkətlərinin həyata keçirilməsi müddətləri", case=False)]

In [6]:
corpus_df['text'].iloc[251274]

'Maddə 12. İcra hərəkətlərinin həyata keçirilməsi müddətləri 12.1. Bu Qanunun 8.2-ci maddəsinin üçüncü bəndində nəzərdə tutulmuş hal istisna olmaqla, icra məmuru icra sənədini aldığı gündən iki ay müddətində bütün zəruri icra hərəkətlərini həyata keçirməlidir. [22] 12.2. Aşağıdakı icra sənədləri dərhal icra olunmalıdır: 12.2.1. alimentlərin, əmək haqqının, habelə bu ödəmələr üzrə bütün borc məbləğlərinin tutulması haqqında; 12.2.2. qanunsuz işdən çıxarılmış və ya başqa işə keçirilmiş işçinin işə və ya əvvəlki vəzifəyə bərpası haqqında. [23] 12.3. Azərbaycan Respublikasının qanunvericiliyi ilə və ya icra sənədi ilə icrası dərhal nəzərdə tutulmuş tələblər təxirəsalınmadan icra olunur. [24]'

## Combined Search

This error message indicates that the request you sent to the OpenAI API has exceeded the maximum allowed tokens per minute (TPM) limit. Specifically:

- **Token Limit Exceeded**: The model has a cap on the number of tokens it can process in a single minute. In your case, the request tried to use 34,449 tokens, exceeding the 30,000 token-per-minute limit for GPT-4.
  
- **Token Calculation**: Tokens are the basic units of text that OpenAI models process, roughly equivalent to words or characters. When building your input, the API counts both input (prompt) and expected output tokens.

To resolve this:
1. **Reduce Input Size**: Truncate or summarize the input prompt to ensure it doesn’t exceed the token limit. For example, you could limit the number of search results or reduce the length of text passages.
2. **Handle Errors**: Use a `try-except` block to handle `RateLimitError` and retry with a smaller input if the error occurs.

You may also need to make multiple smaller API calls if there is too much text to fit within the limit.

In [1]:
import torch
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np
import re
from openai import OpenAI
import tiktoken
from decouple import Config, RepositoryEnv

# Paths to model and corpus file
model_path = r"E:\Software\Data Science and AI\NLP\Edliyye\Legal Acts Question Answering\Fine-Tuning\msmarco-bert-base-dot-v5-cosine\final"
corpus_path = r"E:\Software\Data Science and AI\NLP\Edliyye\Legal Acts Question Answering\Data Collection\corpus_embed.parquet"

# Load the fine-tuned model
model = SentenceTransformer(model_path)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load the embeddings, texts, urls, and names from the corpus file
corpus_df = pd.read_parquet(corpus_path)
embeddings = np.array(corpus_df['embeddings'].tolist())
texts = corpus_df['text'].tolist()
urls = corpus_df['url'].tolist()
names = corpus_df['name'].tolist()

# Specify the path to the .env file
env_path = r"E:\Software\Data Science and AI\NLP\Edliyye\Legal Acts Question Answering\GitHub (Private)\.env"
config = Config(RepositoryEnv(env_path))

# Fetch the OpenAI API key
openai_api_key = config('OPENAI_API_KEY')

# Initialize the OpenAI client using the API key from .env
client = OpenAI(api_key=openai_api_key)

# Define a function to clean the query by removing symbols and numbers at the end
def clean_query(query):
    return re.sub(r'[\d!?.,;]*$', '', query).strip()

# Define a threshold for similarity score
SIMILARITY_THRESHOLD = 0.75  # Adjust the threshold value as needed

# Define a function to compute semantic similarity
def compute_similarity(query, embeddings, top_k=20):
    query_embedding = model.encode(query, convert_to_tensor=False)
    similarities = util.cos_sim(query_embedding, embeddings).cpu().numpy().flatten()
    results = []
    seen_texts = set()
    for i, score in enumerate(similarities):
        if score >= SIMILARITY_THRESHOLD and texts[i] not in seen_texts:
            seen_texts.add(texts[i])
            results.append({
                "query": query,
                "text": texts[i],
                "similarity_score": score,
                "url": urls[i],
                "name": names[i]
            })
    results = sorted(results, key=lambda x: x["similarity_score"], reverse=True)[:top_k]
    return results

# Function to count tokens
def count_tokens(text, model="gpt-4o-2024-08-06"):
    tokenizer = tiktoken.encoding_for_model(model)
    return len(tokenizer.encode(text))

# Define the maximum allowed tokens for the input (adjust based on your limits)
MAX_TOKENS = 30_000

# Define the main function for handling searches and generating the final response
def handle_user_query(user_input):
    # Clean the query
    cleaned_query = clean_query(user_input)

    # Set initial values for top_k and unlimited text_search_limit
    top_k = 20
    text_search_limit = None  # Start with unlimited search limit

    # Try to generate a prompt and catch RateLimitError
    while True:
        try:
            # Conduct text search with initial unlimited limit
            text_search_results = corpus_df[corpus_df['text'].str.contains(cleaned_query, case=False)]
            if text_search_limit is not None:
                text_search_results = text_search_results[:text_search_limit]

            # Run similarity search with current top_k and store results in a DataFrame
            similarity_results = compute_similarity(cleaned_query, embeddings, top_k=top_k)
            similarity_df = pd.DataFrame(similarity_results)

            # Combine results and remove duplicates based on 'text', 'url', and 'name' columns only
            combined_results = pd.concat([text_search_results, similarity_df]).drop_duplicates(subset=['text', 'url', 'name'])

            # Construct the prompt with combined results
            prompt = f"User Query: {cleaned_query}\n\nRetrieved Text Passages:\n\n"
            for _, row in combined_results.iterrows():
                prompt += f"Document Name: {row['name']}\nText: {row['text']}\nURL: {row['url']}\n" + "-" * 50 + "\n"

            # Count tokens in the prompt
            token_count = count_tokens(prompt)

            # Check if token count is within the limit
            if token_count <= MAX_TOKENS:
                print(f"Final token count: {token_count}")
                print()
                break  # Exit the loop if within limit

            # If not within limit, set text_search_limit if not set, then reduce both parameters
            print(f"Token count {token_count} exceeds limit. Reducing top_k and applying text_search_limit...")
            top_k = max(5, top_k - 5)
            if text_search_limit is None:
                text_search_limit = 10  # Apply an initial limit if previously unlimited
            else:
                text_search_limit = max(5, text_search_limit - 5)

        except openai.error.RateLimitError as e:
            print(f"Rate limit exceeded: {e}. Adjusting input and retrying...")

    response = client.chat.completions.create(
        model="gpt-4o-2024-08-06",
        messages=[
            {"role": "system",
             "content": "You are an Azerbaijani lawyer who provides legal advice strictly based on Azerbaijani legislation. Always respond in Azerbaijani. When answering questions, refer to relevant legal acts by the URL and their specific clauses, and provide clear references to the exact articles or sections that apply to the user's query. If there are no relevant text passages among retrieved ones, discard them and base your answer on your own to address the user query!"},
            {"role": "user", "content": prompt}
        ]
    )

    # Extract and return the final response
    gpt_response = response.choices[0].message.content
    return gpt_response

# Test with user input
user_query = input("Sorğunuzu daxil edin: ")
print()
final_response = handle_user_query(user_query)
print(final_response)

E:\Apps\python_envs\cuda_gpu_env\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
You try to use a model that was created with version 3.2.1, however, your version is 3.0.1. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





Sorğunuzu daxil edin: Torpaqlar işğaldan azad edildikdə bunun implications nə ola bilər



E:\Apps\python_envs\cuda_gpu_env\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Final token count: 651

Torpaqlar işğaldan azad edildikdə bir neçə hüquqi və iqtisadi nəticələr ortaya çıxa bilər. Bu nəticələr arasında iqtisadi inkişafın təşviq edilməsi, mülkiyyət hüquqlarının bərpası və ərazilərin yenidən qurulması aspektləri mühüm yer tutur. İşğaldan azad edilmiş ərazilərdə fəaliyyət göstərən sahibkarlar üçün dövlət tərəfindən müəyyən maliyyə resurslarına çıxış imkanları təmin edilir. Belə ki, dövlət zəmanəti ilə kreditlərin verilməsi və kredit faizlərinin subsidiyalaşdırılması kimi mexanizmlər tətbiq edilə bilər. Bu barədə daha ətraflı "İşğaldan azad edilmiş ərazilərdə fəaliyyət göstərən sahibkarlıq subyektlərinə verilən kreditlərin dövlət zəmanəti ilə təmin edilməsi və kredit faizlərinin subsidiyalaşdırılması Qaydası"nda məlumat verilir. Siz [bu keçiddən](https://e-qanun.az/framework/53147) bu qaydalar haqqında daha ətraflı oxuya bilərsiniz. 

Həmçinin, işğaldan azad edilmiş ərazilərdə mülkiyyət hüquqlarının bərpası məsələləri də önə çəkilir. Bu çərçivədə, ərazi

In [4]:
# Test with user input
user_query = input("Sorğunuzu daxil edin: ")
print()
final_response = handle_user_query(user_query)
print(final_response)

Sorğunuzu daxil edin: Azərbaycan Respublikasının torpaqlarının erməni işğalından azad edilməsinin hüquqi nəticələri hansılardır?

Final token count: 2462

Azərbaycan torpaqlarının erməni işğalından azad edilməsinin hüquqi nəticələri Azərbaycan Respublikasının qanunvericilik aktlarına və beynəlxalq hüquq normalarına əsaslanır. Bu nəticələr arasında aşağıdakıları qeyd etmək olar:

1. **Ərazi bütövlüyünün bərpası və dövlət suverenliyinin tam təmin edilməsi**. Öz ərazilərinin idarə olunmasının və bu ərazilərin inkişafının tam şəkildə Azərbaycanın qanuni hakimiyyəti tərəfindən həyata keçirilməsi təmin olunacaq.

2. **Qaçqınların və məcburi köçkünlərin öz yurdlarına qayıtması**. İşğaldan azad edilmiş ərazilərə yenidən qayıdan məcburi köçkünlərin yaşayış şəraitinin yaxşılaşdırılması və bu ərazilərdə təsərrüfat fəaliyyətinin bərpası üçün müxtəlif dövlət səviyyəsində proqramlar hazırlanır və həyata keçirilir. Qaçqınların və məcburi köçkünlərin yaşayış şəraitinin yaxşılaşdırılması və məşğulluğun

In [4]:
# Test with user input
user_query = input("Sorğunuzu daxil edin: ")
print()
final_response = handle_user_query(user_query)
print(final_response)

Sorğunuzu daxil edin: İcra hərəkətlərinin həyata keçirilməsi müddətləri?

Final token count: 2567
İcra hərəkətlərinin həyata keçirilməsi müddətləri barədə Azərbaycan Respublikasının qanunvericiliyinə əsasən, "İcra haqqında" Azərbaycan Respublikası Qanununun 12-ci maddəsinə əsasən icra məmuru icra sənədini aldığı gündən iki ay müddətində bütün zəruri icra hərəkətlərini həyata keçirməlidir. Bu müddət yalnız istisna hallardan başqa bütün icra sənədlərinə aiddir. 

Maddə 12.2. bəzi xüsusi icra sənədlərinin dərhal icra edilməsini nəzərdə tutur, bunlara alimentlərin, əmək haqqının və qanunsuz işdən çıxarılmış işçinin işə bərpası haqqında əmrlər daxildir. Həmin sənədlər dərhal icra olunmalıdır.

Daha ətraflı məlumat əldə etmək üçün "İcra haqqında" Qanunun tam mətni ilə tanış ola bilərsiniz: [İcra haqqında Qanun](https://e-qanun.az/framework/1406).


## Database Preprocessing

In [1]:
import pandas as pd
import sqlite3

# Load the Parquet file
parquet_file_path = r'E:\Software\Data Science and AI\NLP\Edliyye\Legal Acts Question Answering\Data Collection\finetune_data.parquet'
parquet_df = pd.read_parquet(parquet_file_path)

# Connect to the SQLite database
sqlite_db_path = r'E:\Software\Data Science and AI\NLP\Edliyye\Legal Acts Question Answering\Data Collection\db.sqlite3'
conn = sqlite3.connect(sqlite_db_path)

In [5]:
# Load existing data from SQLite
sqlite_df = pd.read_sql_query("SELECT query_text, gpt_response FROM ai_queries_userquery", conn)
display(sqlite_df.head())
display(parquet_df.head())

,query_text,gpt_response
0,miras,Miras hüququ ilə bağlı suallarınıza cavab verə...
1,mirasda məcburi pay,Azərbaycan Respublikası Mülki Məcəlləsinə əsas...
2,informasiya təhlükəsizliyi,Azərbaycan Respublikasının qanunvericiliyinə g...
3,valideyn hüququ,"Ailə Məcəlləsinə (AM) əsasən, valideyn hüquqla..."
4,mirasda məcburi pay,Azərbaycan Respublikasının Mülki Məcəlləsinə ə...


,query,text,label
0,etibarnamə,"Etibarnamə, bir şəxsin (etibar edən) digər şəx...",1.0
1,etibarnamə,Maddə 363. Etibarnamənin müddəti 363.1. Bu Məc...,1.0
2,i̇stehlakcı haqqında,Maddə 452. Fiziki şəxslərə dərman vasitələrini...,1.0
3,i̇stehlakcı haqqında,Maddə 746-2. İstehlak krediti müqaviləsinin ba...,1.0
4,i̇stehlakcı haqqında,Maddə 1071. İstiqraz şəklində borc öhdəliyində...,0.0


In [9]:
display(parquet_df['query'].unique())
len(parquet_df['query'].unique())

array(['etibarnamə', 'i̇stehlakcı haqqında', 'vətəndaşlıq',
       'vətəndaş hüquqları', 'məhkəmə', 'mülki iş', 'iddia müddəti',
       'kassasiya şikayəti', 'müqavilə', 'aliment', 'uşaq hüququ',
       'nikah', 'qəyyumluq', 'valideyn hüququ', 'cinayət məcəlləsi',
       'cinayət təqibi nədir', 'analıq məzuniyyəti',
       'ər-arvadın birgə mülkiyyəti',
       'vəsiyyatnamə üzrə vərəsəliyin açılması',
       'qeyri hökumət təşkilatının ləğvi necə olur?', 'boşanma',
       'yol nişanını xətti pozmağa görə cərimə', 'nikahın ləğvi',
       'lizinq obyektinin geri qaytarilmasi', 'əqdin etibarsizligi',
       'mirasda mecburi pay', 'ekspert rəyi sübut hesab olunurmu?',
       'məhkəməyə müraciət qaydası', 'qəsdən adam öldürmə',
       'alimentin ödənilməsi', 'girov',
       'notarial qaydada təsdiqlənməsi məcburi olan müqavilələrin siyahısı',
       'istintaqın aparılması', 'vərəsəlik hüququ', 'yaşamaq hüququ',
       'birgə mülkiyyət', 'hüquqi şəxsin yenidən təşkili',
       'prokurorluq o

65

In [ ]:
# Find new rows to add (exclude duplicates)
new_data = parquet_df.merge(sqlite_df, on=['query_text', 'gpt_response'], how='left', indicator=True)
new_data = new_data[new_data['_merge'] == 'left_only'].drop(columns=['_merge'])

# Append new rows to SQLite
new_data.to_sql('ProductionQuery', conn, if_exists='append', index=False)

print(f"Added {len(new_data)} new rows to the database.")

In [10]:
# Close the connection
conn.close()

## Preprocessing

In [ ]:
import pandas as pd

corpus_path = r"E:\Software\Data Science and AI\NLP\Edliyye\Legal Acts Question Answering\Data Collection\corpus_embed.parquet"

df = pd.read_parquet(corpus_path)

display(df)
df.info()
df.index

### Filter text passages where there is no words or only numbers

In [ ]:
df.loc[df['text'] == '– – – – digərləri: 4407 29610 0 – – – –']

In [ ]:
# Define a regex pattern to match entries with exactly one word, mixed with numbers and symbols
pattern = r'^(?:[^\w]*\b\w+\b[^\w\d\s]*)?(?:[\d.,–:\s-]*\b\w+\b[\d.,–:\s-]*){1}$'

# Filter out rows that match the pattern
df = df[~df['text'].str.contains(pattern, regex=True)]

# Reset the index
df = df.reset_index(drop=True)

display(df)
df.info()
df.index

In [ ]:
# Define a regex pattern to match rows with only numbers and symbols
pattern = r'^[\d\s.,–-]*$'

# Filter out rows that match the pattern
df = df[~df['text'].str.contains(pattern)]

# Reset the index
df = df.reset_index(drop=True)

display(df)
df.info()
df.index

In [ ]:
# Define a regex pattern to match rows with only numbers, dots, commas, or dashes (and possibly spaces between them)
pattern = r'^(?:[\d.,\-\s]+)$'

# Filter out rows that match the pattern
df = df[~df['text'].str.contains(pattern)]

# Reset the index
df = df.reset_index(drop=True)

display(df)
df.info()
df.index

### Keyword-Based Search

In [ ]:
# Lowercase all text
df['text'] = df['text'].str.lower()

In [ ]:
s = df.loc[df['text'].str.contains("icra hərəkətləri", case=False)]
s

In [ ]:
s.loc[s['name'] == 'İcra haqqında']

In [ ]:
df['text'].iloc[251274]

In [ ]:
["icra hərəkətləri", "basa çatdırılma müddəti", "icra müddəti", "icra prosesi", "müddət tələbləri", "icra vaxtı"]

In [ ]:
df.to_parquet(corpus_path, index=False)

In [ ]:
query = 'icra hərəkətləri neçə günə başa çatdırılmalıdır?'
query

In [ ]:
query_embedding = model.encode(query, device=device)

In [ ]:
text_embeddings = corpus_df['embeddings']

In [ ]:
# Define a threshold for similarity score (e.g., 0.5)
SIMILARITY_THRESHOLD = 0.5

# Function to compute similarity scores with batching and optimized cosine similarity
def compute_similarity(query, texts, threshold=SIMILARITY_THRESHOLD):    
    # Compute cosine similarities between the query and all texts at once
    cosine_scores = util.cos_sim(query_embedding, text_embeddings).cpu().numpy().flatten()
    
    # Collect results that meet the threshold
    results = [
        {"query": query, "text": texts[i], "similarity_score": score}
        for i, score in enumerate(cosine_scores) if score >= threshold
    ]

    # Sort results by similarity score in descending order
    results = sorted(results, key=lambda x: x["similarity_score"], reverse=True)
    return results

# Run similarity computation
results = compute_similarity(query, texts)

# Display sorted results that meet the threshold
for result in results:
    print(f"Query: {result['query']}")
    print(f"Text: {result['text']}")
    print(f"Similarity Score: {result['similarity_score']:.4f}")
    print("-" * 50)

In [ ]:
from sentence_transformers import SentenceTransformer, util

cosine_score = util.cos_sim(query_embedding, text_embedding).item()

In [ ]:
cosine_score

In [ ]:
corpus_df['text'].iloc[277320]

## Model's Tokenizer Check

In [ ]:
from sentence_transformers import SentenceTransformer

# Load the pre-trained model
model = SentenceTransformer("sentence-transformers/msmarco-bert-base-dot-v5")

# Access the tokenizer
tokenizer = model.tokenizer

# Define sample texts
texts = ["cinayət təqibi nədir?", "cinayət məsuliyyəti", "işgəncə vermək", "QARABAĞA QAYIDIŞ!"]

# Tokenize each text and display tokens
for text in texts:
    tokens = tokenizer.tokenize(text)
    print(f"Text: {text}")
    print(f"Tokens: {tokens}")
    print("\n")

## Preprocessing the Corpus

In [ ]:
import pandas as pd

path = r"E:\Software\Data Science and AI\NLP\Edliyye\Legal Acts Question Answering\Data Collection\corpus.parquet"
df = pd.read_parquet(path)

display(df)
df.info()
df.index

In [ ]:
# Calculate word count for each row in the 'text' column
#df['word_count'] = df['text'].apply(lambda x: len(x.split()))

# Display basic statistics for the word count column
word_count_stats = df['word_count'].describe()
display(word_count_stats)

In [ ]:
# Remove rows where word count is less than 7
#df = df.loc[df['word_count'] > 7].reset_index(drop=True)

In [ ]:
df.loc[df['word_count'] <= 10]

In [ ]:
df['text'].iloc[523333]

In [ ]:
df.loc[df['text'] == '']

Referendum aktları
Konstitusiya
AZƏRBAYCAN RESPUBLİKASININ KONSTİTUSİYA QANUNLARI
Məcəllələr
Qanunlar

AZƏRBAYCAN RESPUBLİKASI NAZİRLƏR KABİNETİNİN QƏRARLARI
AZƏRBAYCAN RESPUBLİKASI PREZİDENTİNİN FƏRMANLARI 

In [ ]:
# Locate entries with the pattern "– çıxarılmışdır" in the text column
df.loc[df['text'].str.contains(r"ləğv edilmişdir", regex=True)]

In [ ]:
df['text'].iloc[133057]

In [ ]:
df.loc[df['word_count'] > 10000]

In [ ]:
df.iloc[30265]

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize

# Ensure you have the sentence tokenizer data
#nltk.download('punkt_tab')

# Sample the first 5 rows of the DataFrame
#sample_df = df.head(5)

# Initialize an empty list to store the results
expanded_rows = []

# Iterate over each row
for _, row in df.iterrows():
    text = row['text']
    sentences = sent_tokenize(text)  # Split text into sentences
    temp_text = []  # Temporary list to accumulate sentences
    word_count = 0  # Initialize word count for the current segment
    
    for sentence in sentences:
        # Calculate word count of the sentence
        sentence_word_count = len(sentence.split())
        
        # Check if adding this sentence will exceed the 100-word limit
        if word_count + sentence_word_count > 100:
            # Append the current segment as a new row
            new_row = row.copy()  # Copy original row data
            new_row['text'] = ' '.join(temp_text)  # Join accumulated sentences
            expanded_rows.append(new_row)
            
            # Reset for the next segment
            temp_text = []
            word_count = 0
        
        # Add the sentence to the current segment
        temp_text.append(sentence)
        word_count += sentence_word_count
    
    # Add any remaining sentences as a new row
    if temp_text:
        new_row = row.copy()
        new_row['text'] = ' '.join(temp_text)
        expanded_rows.append(new_row)

# Create a new DataFrame from the expanded rows
expanded_df = pd.DataFrame(expanded_rows)

# Remove rows with empty strings in the 'text' column
expanded_df = expanded_df[expanded_df['text'].str.strip() != '']

# Display the new DataFrame
expanded_df.reset_index(drop=True, inplace=True)

expanded_df['word_count'] = expanded_df['text'].apply(lambda x: len(x.split()))
display(expanded_df['word_count'].describe())

display(expanded_df)
expanded_df.info()

In [ ]:
# Define the function to split text into 100-word segments
def split_into_segments(text, segment_size=100):
    words = text.split()
    return [' '.join(words[i:i+segment_size]) for i in range(0, len(words), segment_size)]

# Apply the function and explode in-place if word count is 400 or more
df['text'] = df.apply(lambda row: split_into_segments(row['text']) if row['word_count'] >= 400 else row['text'], axis=1)
df = df.explode('text', ignore_index=True)  # Explode to get each segment as a new row

# Recalculate word count if needed
df['word_count'] = df['text'].apply(lambda x: len(x.split()))

# Display the result
df.reset_index(drop=True, inplace=True)

display(df)
df.info()
df.index

word_count_stats = df['word_count'].describe()
display(word_count_stats)

In [ ]:
display(df['text'].iloc[0])
display(expanded_df['text'].iloc[0])
display(expanded_df['text'].iloc[1])
display(expanded_df['text'].iloc[2])
display(expanded_df['text'].iloc[3])
display(expanded_df['text'].iloc[4])
display(expanded_df['text'].iloc[5])
display(expanded_df['text'].iloc[6])

display(expanded_df['text'].iloc[7])
display(df['text'].iloc[1])

In [ ]:
display(expanded_df['text'].iloc[11])
display(expanded_df['text'].iloc[12])
display(df['text'].iloc[2])

In [ ]:
expanded_df.loc[expanded_df['text'] == '']

In [ ]:
# Display basic statistics for the word count column
word_count_stats = expanded_df['word_count'].describe()
display(word_count_stats)

In [ ]:
df.to_parquet(path, index=False)

In [ ]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split

from tokenizers import Tokenizer
from transformers import BertTokenizerFast
from transformers import BertTokenizer, BertForMaskedLM, BertConfig, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
from datasets import Dataset as HFDataset

In [ ]:
# Define the core path
PATH_DATA = 'E:/Software/Data Science and AI/NLP/Edliyye/Legal Acts Question Answering/NLP project'
PATH_VOCAB = 'E:/Software/Data Science and AI/NLP/Edliyye/Legal Acts Question Answering/GitHub (Private)/tokenizer_directory'

text_path = os.path.join(PATH_DATA, 'full_qanun_text_chunks.parquet')

## Tokenizer and Model Setup

In [ ]:
# Load the pre-trained BERT tokenizer
base_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased') # Originally vocab_size=105_879

# Load your custom tokenizer
custom_tokenizer = BertTokenizer.from_pretrained(PATH_VOCAB) # vocab_size=80_342

# Correct model_max_length if necessary
custom_tokenizer.model_max_length = 512

# Add new tokens from custom tokenizer to base tokenizer
new_tokens = set(custom_tokenizer.get_vocab().keys()) - set(base_tokenizer.get_vocab().keys())
base_tokenizer.add_tokens(list(new_tokens))

# Load the pre-trained model
model = BertForMaskedLM.from_pretrained('bert-base-multilingual-uncased')

# Resize the model's embeddings to accommodate new tokens
model.resize_token_embeddings(len(base_tokenizer))

In [ ]:
len(new_tokens)

## Load and Prepare the Dataset

In [ ]:
# Load the Azerbaijani corpus
df = pd.read_parquet(text_path)

# Extract the text column
texts = df['text'].tolist()

In [ ]:
# Split the data into training and validation sets
train_texts, val_texts = train_test_split(texts, test_size=0.2)

# Load the pre-trained BERT tokenizer
base_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')  # Originally vocab_size=105_879

# Load your custom tokenizer
custom_tokenizer = BertTokenizer.from_pretrained(PATH_VOCAB)  # vocab_size=80_342

# Ensure the custom tokenizer has a reasonable max length
custom_tokenizer.model_max_length = 512

# Add new tokens from custom tokenizer to base tokenizer
new_tokens = set(custom_tokenizer.get_vocab().keys()) - set(base_tokenizer.get_vocab().keys())
base_tokenizer.add_tokens(list(new_tokens))

# Load the pre-trained model
model = BertForMaskedLM.from_pretrained('bert-base-multilingual-uncased')

# Resize the model's embeddings to accommodate new tokens
model.resize_token_embeddings(len(base_tokenizer))

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Ensure the model is on GPU
print("Model device:", next(model.parameters()).device)

# Tokenize the dataset
def tokenize_and_encode(texts):
    input_ids, attention_masks = [], []
    for text in texts:
        encoded = base_tokenizer.encode_plus(
            text,
            max_length=512,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0)

train_input_ids, train_attention_masks = tokenize_and_encode(train_texts)
val_input_ids, val_attention_masks = tokenize_and_encode(val_texts)

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_masks):
        self.input_ids = input_ids
        self.attention_masks = attention_masks

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_masks[idx],
        }

train_dataset = CustomDataset(train_input_ids, train_attention_masks)
val_dataset = CustomDataset(val_input_ids, val_attention_masks)

# Data collator for MLM
data_collator = DataCollatorForLanguageModeling(
    tokenizer=base_tokenizer,
    mlm=True,
    mlm_probability=0.15
)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
    dataloader_num_workers=4,
    report_to="none"  # To disable logging reports
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Fine-tune the model
trainer.train()

In [ ]:
# Save the fine-tuned model
model.save_pretrained('./fine-tuned-model')
base_tokenizer.save_pretrained('./fine-tuned-model')

In [ ]:
# Save the fine-tuned model
model.save_pretrained(f'{PATH}/fine-tuned-model')
base_tokenizer.save_pretrained(f'{PATH}/fine-tuned-model')

## Dataset Class Definition

In [ ]:
class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=512):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return encoding.input_ids[0] # Explicitly move the data to the GPU

## Data Collator and DataLoader

In [ ]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Create the dataset
dataset = TextDataset(texts, base_tokenizer)

# Define a custom data collator to ensure tensors are on the correct device
class DataCollatorWithDevice:
    def __init__(self, data_collator, device):
        self.data_collator = data_collator
        self.device = device

    def __call__(self, examples):
        batch = self.data_collator(examples)
        # Move batch to the correct device
        batch['input_ids'] = batch['input_ids'].to(self.device)
        batch['attention_mask'] = batch['attention_mask'].to(self.device)
        if 'labels' in batch:
            batch['labels'] = batch['labels'].to(self.device)
        return batch

# Create the data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=base_tokenizer,
    mlm=True,
    mlm_probability=0.15
)

# Wrap the data collator with the custom collator to handle device placement
data_collator_with_device = DataCollatorWithDevice(data_collator, device)

# Create the dataloader
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, collate_fn=data_collator_with_device)

## Training Setup

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
    dataloader_num_workers=4  # To speed up data loading
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator_with_device,
    train_dataset=dataset
)

# Fine-tune the model
trainer.train()

# Verify CUDA Availability
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))

In [ ]:
# Save the fine-tuned model
model.save_pretrained(f'{PATH}/fine-tuned-model')
base_tokenizer.save_pretrained(f'{PATH}/fine-tuned-model')

## Load and Use the Fine-Tuned Model

In [ ]:
MODEL_PATH = os.path.join(PATH, 'fine-tuned-model')

# Load the model and tokenizer
model = BertForMaskedLM.from_pretrained(MODEL_PATH)
tokenizer = BertTokenizer.from_pretrained(MODEL_PATH)